In [1]:
from transformers import AutoTokenizer, LlamaConfig, LlamaTokenizer,LlamaForCausalLM, LlamaTokenizer,TextDataset,DataCollatorForLanguageModeling
from peft import get_peft_model, PeftModel, PeftConfig ,LoraConfig,TaskType
from transformers import TrainingArguments, Trainer,GenerationConfig,LineByLineTextDataset
from datasets import Dataset , load_dataset
import torch
import accelerate
from transformers.generation.utils import GreedySearchDecoderOnlyOutput
import textwrap
#import bitsandbytes
from torch.nn import CrossEntropyLoss
import numpy as np

/home/minigpt_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/minigpt_env/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/minigpt_env/lib/libcudart.so'), PosixPath('/home/minigpt_env/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [2]:
#torch.cuda.empty_cache()


In [3]:
path="/home/vicuna-weights-7B"
tokenizer = LlamaTokenizer.from_pretrained(path)
tokenizer.pad_token_id = ( 0)
tokenizer.padding_side = "left"

In [4]:
max_len=256

In [5]:
def generate_prompt(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""
 
 
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=max_len,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < max_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
 
    result["labels"] = result["input_ids"].copy()
 
    return result
 
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

In [6]:
block_size = 128
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="/home/medical-llama/medical_train.txt",
    block_size=block_size,)

/home/minigpt_env/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [7]:
block_size = 128
eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="/home/medical-llama/medical_val.txt",
    block_size=block_size,)

In [8]:
"""
with open("/home/medical-llama/cleaned_medical_news.txt", "r", encoding="utf-8") as file:
    medical_text = file.read()

tokenized_text = tokenizer.encode(medical_text)
len(tokenized_text)
"""

'\nwith open("/home/medical-llama/cleaned_medical_news.txt", "r", encoding="utf-8") as file:\n    medical_text = file.read()\n\ntokenized_text = tokenizer.encode(medical_text)\nlen(tokenized_text)\n'

In [9]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False,
) # handles next word prediction task

In [10]:
torch.set_default_dtype(torch.half)
model = LlamaForCausalLM.from_pretrained("/home/vicuna-weights-7B")
torch.set_default_dtype(torch.float)

Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.98s/it]


In [11]:

#model = model.eval()
#model = torch.compile(model)

In [12]:
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM,
 inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1,)
model = get_peft_model(model, peft_config).to('cuda:0')
model.print_trainable_parameters()

trainable params: 4194304 || all params: 6742618112 || trainable%: 0.06220586618327525


In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    ignore_index = -100
    loss_fct = CrossEntropyLoss(ignore_index=ignore_index)

    logits = logits.view(-1, logits.shape[-1])
    labels = labels.view(-1)
    loss = loss_fct(logits, labels)
    perplexity = np.exp(loss.item())
    return {"perplexity": perplexity}

In [14]:
training_args = TrainingArguments(
    
    output_dir="~/medical-llama/output-1",
    overwrite_output_dir=True,
    logging_dir="~/medical-llama/logs-news",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    save_steps=600,
    save_total_limit=3,
    fp16=True,
    warmup_ratio=0.02,
    weight_decay=0.01,
    learning_rate=0.00005,
    logging_steps=100,
    evaluation_strategy='steps',
    eval_steps=200,
    
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
     
)

In [16]:
model = torch.compile(model)

In [17]:

trainer.train()

/home/minigpt_env/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: deelipvenkat. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
200,1.719400,1.803088
400,1.659900,1.766203
600,1.626400,1.752393
800,1.639800,1.745714
1000,1.627100,1.739460
1200,1.598100,1.735234


TrainOutput(global_step=4884, training_loss=1.6013463463185753, metrics={'train_runtime': 5119.377, 'train_samples_per_second': 7.631, 'train_steps_per_second': 0.954, 'total_flos': 3.96728064609878e+17, 'train_loss': 1.6013463463185753, 'epoch': 3.0})

In [ ]:
PROMPT_TEMPLATE = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.
 
### Instruction:
[INSTRUCTION]
 
### Response:
"""

def create_prompt(instruction: str) -> str:
    return PROMPT_TEMPLATE.replace("[INSTRUCTION]", instruction)


def generate_response(prompt: str, model: PeftModel) -> GreedySearchDecoderOnlyOutput:
    encoding = tokenizer(prompt, return_tensors="pt")
    input_ids = encoding["input_ids"].to('cuda')
 
    generation_config = GenerationConfig(
        temperature=0.5,
        top_p=0.75,
        repetition_penalty=1.1,
        beam_size=1,
        max_length=128,
        min_length=1,
        
    )
    with torch.inference_mode():
        return model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=128,
        )
    

def format_response(response: GreedySearchDecoderOnlyOutput) -> str:
    decoded_output = tokenizer.decode(response.sequences[0])
    response = decoded_output.split("### Response:")[1].strip()
    return "\n".join(textwrap.wrap(response))


def ask_vicuna(prompt: str, model=model) -> str:
    prompt = create_prompt(prompt)
    response = generate_response(prompt, model)
    b=format_response(response)
    a=b.split('###')
    return a[0]

In [1]:
question="How can we enhance the efficiency and scalability of manufacturing processes for cell and gene therapies to make them more accessible and affordable?"

In [2]:
question="""
I want datasets which are either in txt or json format which contains medical or bio-tech text data.
"""
print(ask_vicuna(question))

NameError: name 'ask_vicuna' is not defined

In [18]:
#model.save_pretrained("/home/medical-llama/saved_models-news")